In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np

from tensorflow.keras import layers
from functools import partial

tf.random.set_seed(42)
np.random.seed(42)

In [2]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
((X_train, y_train), (X_test, y_test)) = keras.datasets.cifar10.load_data()
X_train = X_train.astype("float")
X_test = X_test.astype("float")

In [4]:
mean = np.mean(X_train, axis=0)
X_train -= mean
X_test -= mean

In [5]:
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)
y_test = lb.transform(y_test)

In [6]:
aug = keras.preprocessing.image.ImageDataGenerator(
    height_shift_range=0.1,
    width_shift_range=0.1,
    zoom_range=0.1,
    rotation_range=0.1,
    horizontal_flip=True, 
    fill_mode="nearest",
)

In [7]:
def train_model(name, create_model, optimizer=keras.optimizers.Nadam()):
    keras.backend.clear_session()
    model = create_model(X_train.shape[1:], y_train.shape[1])

    model.compile(
        metrics=["accuracy"],
        optimizer=optimizer,
        loss=keras.losses.categorical_crossentropy
    )

    callbacks = [
        keras.callbacks.ModelCheckpoint(f"{name}.h5", monitor="val_accuracy", verbose=1, save_best_only=True),
        keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=5, verbose=1),
    ]

    history = model.fit(
        aug.flow(X_train, y_train, 64),
        validation_data=(X_test, y_test),
        steps_per_epoch=len(X_train)//64,
        epochs=200,
        callbacks=callbacks,
        verbose=1
    )

    return keras.models.load_model(f"{name}.h5")

In [8]:
Conv2D = partial(
    layers.Conv2D,
    padding="SAME",
    activation="relu",
    strides=1,
)
MaxPool2D = partial(
    layers.MaxPool2D,
    padding="SAME",
    strides=1
)

def interception_block(input_layer, l1, l2_2, l3_2, l4_2, l2_1, l3_1):
    l1 = Conv2D(filters=l1, kernel_size=1)(input_layer)

    l2_1 = Conv2D(filters=l2_1, kernel_size=1)(input_layer)
    l2_2 = Conv2D(filters=l2_2, kernel_size=3)(l2_1)

    l3_1 = Conv2D(filters=l3_1, kernel_size=1)(input_layer)
    l3_2 = Conv2D(filters=l3_2, kernel_size=5)(l3_1)

    l4_1 = MaxPool2D(pool_size=3)(input_layer)
    l4_2 = Conv2D(filters=l4_2, kernel_size=1)(l4_1)

    return layers.concatenate([l1, l2_2, l3_2, l4_2], axis=-1)

In [9]:
LocalRespNorm = partial(
    layers.Lambda,
    tf.nn.local_response_normalization
)

In [10]:
AvgPool2D = partial(
    layers.AvgPool2D,
    padding="VALID",
    strides=1
)


def GoogleNet(input_shape, output_shape):
    input_layer = layers.Input(shape=input_shape)

    X = Conv2D(filters=64, kernel_size=7, strides=2)(input_layer)
    X = MaxPool2D(pool_size=3, strides=2)(X)
    X = LocalRespNorm()(X)

    X = Conv2D(filters=64, kernel_size=1, padding="VALID")(X)
    X = Conv2D(filters=192, kernel_size=3)(X)
    X = LocalRespNorm()(X)

    X = MaxPool2D(pool_size=3, strides=2)(X)
    X = interception_block(X, 64, 128, 32, 32, 96, 16)
    X = interception_block(X, 128, 192, 96, 64, 128, 32)

    X = MaxPool2D(pool_size=3, strides=2)(X)
    X = interception_block(X, 192, 208, 48, 64, 96, 16)

    # extra network 1
    X1 = AvgPool2D(pool_size=5, strides=3)(X)
    X1 = Conv2D(filters=128, kernel_size=1)(X1)
    X1 = layers.Flatten(X1)
    X1 = layers.Dense(1024, activation="relu")(X1)
    X1 = layers.Dropout(0.7)(X1)
    X1 = layers.Dense(5, activation="softmax")(X1)

    X = interception_block(X, 160, 224, 64, 64, 112, 24)
    X = interception_block(X, 128, 256, 64, 64, 128, 24)
    X = interception_block(X, 112, 288, 64, 64, 144, 32)

    # extra network 2
    X2 = AvgPool2D(pool_size=5, strides=3)(X)
    X2 = Conv2D(filters=128, kernel_size=1)(X2)
    X2 = layers.Flatten()(X2)
    X2 = layers.Dense(1024, activation="relu")(X2)
    X2 = layers.Dropout(0.7)(X2)
    X2 = layers.Dense(output_shape, activation="softmax")(X2)

    X = interception_block(X, 256, 320, 128, 128, 160, 32)
    X = MaxPool2D(pool_size=3, strides=2)(X)
    X = interception_block(X, 256, 320, 128, 128, 160, 32)
    X = interception_block(X, 384, 384, 128, 128, 192, 48)

    X = layers.GlobalAvgPool2D()(X)
    X = layers.Dropout(0.4)(X)
    X = layers.Dense(output_shape, activation="softmax")(X)

    model = keras.models.Model(input_layer, [X, X1, X2], name="GoogLeNet")
    return model

In [11]:
def vgg16(input_shape, output_shape):
    input_layer = layers.Input(input_shape)
    
    X = Conv2D(filters=64, kernel_size=3)(input_layer)
    X = Conv2D(filters=64, kernel_size=3)(X)
    X = MaxPool2D(pool_size=2, strides=2)(X)
    X = Conv2D(filters=128, kernel_size=3)(X)
    X = Conv2D(filters=128, kernel_size=3)(X)
    X = MaxPool2D(pool_size=2, strides=2)(X)
    X = Conv2D(filters=256, kernel_size=3)(X)
    X = Conv2D(filters=256, kernel_size=3)(X)
    X = Conv2D(filters=256, kernel_size=3)(X)
    X = MaxPool2D(pool_size=2, strides=2)(X)
    X = Conv2D(filters=512, kernel_size=3)(X)
    X = Conv2D(filters=512, kernel_size=3)(X)
    X = Conv2D(filters=512, kernel_size=3)(X)
    X = MaxPool2D(pool_size=2, strides=2)(X)
    X = Conv2D(filters=512, kernel_size=3)(X)
    X = Conv2D(filters=512, kernel_size=3)(X)
    X = Conv2D(filters=512, kernel_size=3)(X)
    X = MaxPool2D(pool_size=2, strides=2)(X)

    X = layers.Flatten()(X)
    X = layers.Dense(4096, activation="relu")(X)
    X = layers.Dense(4096, activation="relu")(X)
    X = layers.Dense(output_shape, activation="softmax")(X)

    model = keras.models.Model(input_layer, [X], name="vgg16")
    return model 


In [ ]:
vgg16_model = train_model("vgg16", vgg16, optimizer=keras.optimizers.SGD(learning_rate=1e-3, momentum=0.8))

In [14]:
def Conv2D_batch(X, filters, kernel_size, strides=1):
    X = layers.Conv2D(
        filters=filters,
        kernel_size=kernel_size,
        strides=strides,
        padding="SAME",
        use_bias=False
    )(X)
    X = layers.BatchNormalization()(X)
    return X

# deepthwise 
def SepConf2D_batch(X, filters, kernel_size, strides=1):
    X = layers.SeparableConv2D(
        filters=filters,
        kernel_size=kernel_size,
        strides=strides,
        padding="SAME",
        use_bias=False
    )(X)
    X = layers.BatchNormalization()(X)
    return X

def xception(input_shape, output_shape):
    def entry_flow(X):
        X = Conv2D_batch(X, 32, 3, 2)
        X = layers.ReLU()(X)
        X = Conv2D_batch(X, 64, 3, 1)

        # 1st block
        tensor = layers.ReLU()(X)
        X = SepConf2D_batch(tensor, 128, 3)
        X = layers.ReLU()(X)
        X = SepConf2D_batch(X, 128, 3)
        X = layers.MaxPool2D(pool_size=3, strides=2, padding="SAME")(X)
        tensor = Conv2D_batch(tensor, 128, 1, 2)
        X = layers.Add()([tensor, X])

        # 2nd block
        X = layers.ReLU()(X)
        X = SepConf2D_batch(X, 256, 3)
        X = layers.ReLU()(X)
        X = SepConf2D_batch(X, 256, 3)
        X = layers.MaxPool2D(pool_size=3, strides=2, padding="SAME")(X)
        tensor = Conv2D_batch(tensor, 256, 1, 2)
        X = layers.Add()([tensor, X])

        # 3rd block
        X = layers.ReLU()(X)
        X = SepConf2D_batch(X, 728, 3)
        X = layers.ReLU()(X)
        X = SepConf2D_batch(X, 728, 3)
        X = layers.MaxPool2D(pool_size=3, strides=2, padding="SAME")(X)
        tensor = Conv2D_batch(tensor, 728, 1, 2)
        X = layers.Add()([tensor, X])

        return X

    def middle_flow(tensor):
        for _ in range(8): 
            X = layers.ReLU()(tensor)
            X = SepConf2D_batch(X, 728, 3)
            X = layers.ReLU()(X)
            X = SepConf2D_batch(X, 728, 3)
            X = layers.ReLU()(X)
            X = SepConf2D_batch(X, 728, 3)
            tensor = layers.Add()([tensor, X])
        return tensor

    def exit_flow(tensor):
        X = layers.ReLU()(tensor)
        X = SepConf2D_batch(X, 728, 3)
        X = layers.ReLU()(X)
        X = SepConf2D_batch(X, 1024, 3)
        X = layers.MaxPool2D(pool_size=3, strides=2, padding="SAME")(X)

        tensor = Conv2D_batch(tensor, 1024, 1, 2)
        X = layers.Add()([tensor, X])

        X = SepConf2D_batch(X, 1536, 3)
        X = layers.ReLU()(X)
        X = SepConf2D_batch(X, 2048, 3)
        X = layers.GlobalAvgPool2D()(X)

        X = layers.Dense(output_shape, activation="softmax")(X)
        return X

    input = layers.Input(shape=input_shape)
    X = entry_flow(input)
    X = middle_flow(X)
    output = exit_flow(X)

    model = keras.models.Model(inputs=input, outputs=output)
    return model

In [ ]:
xception_model = train_model("xception", xception)

In [16]:
class ResidualUnit(keras.layers.Layer):
    def __init__(self, filters, strides=1, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.main_layers = [
            layers.Conv2D(filters, 3, strides=strides, padding="SAME", use_bias=False),
            layers.BatchNormalization(),
            self.activation,
            layers.Conv2D(filters, 3, strides=1, padding="SAME", use_bias=False), 
            layers.BatchNormalization()
        ]
        self.skip_layers = []

        if strides > 1:
            self.skip_layers = [
                keras.layers.Conv2D(filters, 1, strides=strides, padding="SAME", use_bias=False),
                keras.layers.BatchNormalization()
            ]
    def call(self, inputs):
        z = inputs
        for layer in self.main_layers:
            z = layer(z)
        skip_z = inputs
        for layer in self.skip_layers: 
            skip_z = layer(skip_z)
        return self.activation(z + skip_z)

In [17]:
def ResNet34(input_shape, output_shape):
    input_layer = layers.Input(shape=input_shape)
    X = layers.Conv2D(64, 7, strides=2, padding="SAME", use_bias=False)(input_layer)
    X = layers.BatchNormalization()(X)
    X = layers.Activation("relu")(X)
    X = layers.MaxPool2D(pool_size=3, strides=2, padding="SAME")(X)

    prev_filters = 64
    for filters in [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3:
        strides = 1 if filters == prev_filters else 2
        
        X = ResidualUnit(filters, strides=strides)(X)
        prev_filters = filters

    X = layers.GlobalAvgPool2D()(X)
    X = layers.Flatten()(X)
    output_layer = layers.Dense(output_shape, activation="softmax")(X)

    model = keras.models.Model(input_layer, output_layer)
    return model